In [1]:
#r "nuget:Docker.DotNet"
#r "nuget:Npgsql"

using Docker.DotNet;
using Docker.DotNet.Models;
using Npgsql;
using System.Threading;
using System.Threading.Tasks;
using System.Data.Common;

Installing package Docker.DotNet........................................done!

Successfully added reference to package Docker.DotNet, version 3.125.2

Installing package Npgsql.................done!

Successfully added reference to package Npgsql, version 4.1.1

In [2]:
private async Task<(CreateContainerResponse, string)> GetContainer(DockerClient client, string image, string tag)
{
    var hostPort = new Random((int)DateTime.UtcNow.Ticks).Next(10000, 12000);
    //look for image
    var images = await client.Images.ListImagesAsync(new ImagesListParameters()
    {
        MatchName = $"{image}:{tag}",
    }, CancellationToken.None);

    //check if container exists
    var pgImage = images.FirstOrDefault();
    if (pgImage == null)
        throw new Exception($"Docker image for {image}:{tag} not found.");

    //create container from image
    var container = await client.Containers.CreateContainerAsync(new CreateContainerParameters()
    {
        User = "postgres",
        Env = new List<string>()
        {
            "POSTGRES_PASSWORD=password",
            "POSTGRES_DB=repotest",
            "POSTGRES_USER=postgres"
        },
        ExposedPorts = new Dictionary<string, EmptyStruct>()
        {
            ["5432"] = new EmptyStruct()
        },
        HostConfig = new HostConfig()
        {
            PortBindings = new Dictionary<string, IList<PortBinding>>()
            {
                ["5432"] = new List<PortBinding>()
                    {new PortBinding() {HostIP = "0.0.0.0", HostPort = $"{hostPort}"}}
            }
        },
        Image = $"{image}:{tag}",
    }, CancellationToken.None);

    if (!await client.Containers.StartContainerAsync(container.ID, new ContainerStartParameters()
    {
        DetachKeys = $"d={image}"
    }, CancellationToken.None))
    {
        throw new Exception($"Could not start container: {container.ID}");
    }

    var count = 10;
    Thread.Sleep(5000);
    var containerStat = await client.Containers.InspectContainerAsync(container.ID, CancellationToken.None);
    while (!containerStat.State.Running && count-- > 0)
    {
        Thread.Sleep(1000);
        containerStat = await client.Containers.InspectContainerAsync(container.ID, CancellationToken.None);

    }
    Thread.Sleep(10000); //I need some time for the DB to finish starting up so that my tests don't report the DB is starting up
    return (container, $"{hostPort}");
}

In [3]:
DbConnection Db;
DockerClient _client;
CreateContainerResponse _containerResponse;

_client = new DockerClientConfiguration(new Uri("npipe://./pipe/docker_engine")).CreateClient();



In [4]:
//var (containerRespose, port) = await GetContainer(_client, "postgres", "latest");
//_containerResponse = containerRespose;

In [5]:
var hostPort = new Random((int)DateTime.UtcNow.Ticks).Next(10000, 12000);

var images = await _client.Images.ListImagesAsync(new ImagesListParameters()
    {
    MatchName = "postgres:latest",
    }, CancellationToken.None);

//check if container exists
    var pgImage = images.FirstOrDefault();
    if (pgImage == null)
        throw new Exception($"Docker image for postgres:latest not found.");

    //create container from image
    var container = await _client.Containers.CreateContainerAsync(new CreateContainerParameters()
    {
        User = "postgres",
        Env = new List<string>()
        {
            "POSTGRES_PASSWORD=password",
            "POSTGRES_DB=repotest",
            "POSTGRES_USER=postgres"
        },
        ExposedPorts = new Dictionary<string, EmptyStruct>()
        {
            ["5432"] = new EmptyStruct()
        },
        HostConfig = new HostConfig()
        {
            PortBindings = new Dictionary<string, IList<PortBinding>>()
            {
                ["5432"] = new List<PortBinding>()
                    {new PortBinding() {HostIP = "0.0.0.0", HostPort = $"{hostPort}"}}
            }
        },
        Image = "postgres:latest",
    }, CancellationToken.None);
    
    if (!await _client.Containers.StartContainerAsync(container.ID, new ContainerStartParameters()
    {
        DetachKeys = "d=postgres"
    }, CancellationToken.None))
    {
        throw new Exception($"Could not start container: {container.ID}");
    }

    var count = 10;
    Thread.Sleep(5000);
    var containerStat = await _client.Containers.InspectContainerAsync(container.ID, CancellationToken.None);
    while (!containerStat.State.Running && count-- > 0)
    {
        Thread.Sleep(1000);
        containerStat = await _client.Containers.InspectContainerAsync(container.ID, CancellationToken.None);

    }
    Thread.Sleep(10000); //I need some time for the DB to finish starting up so that my tests don't report the DB is starting up


In [6]:
var connectionStringBuilder = new NpgsqlConnectionStringBuilder() { ConnectionString = $"User ID=postgres;Password=password;Server=127.0.0.1;Port={hostPort};Database=repotest;Integrated Security=true;Pooling=false;CommandTimeout=300" };
Db = new NpgsqlConnection(connectionStringBuilder.ConnectionString);
Db.Open();

return String.Equals("repotest", Db.Database);

Db.Close(); 
Db.Dispose();
//stop container
if (await _client.Containers.StopContainerAsync(_containerResponse.ID, new ContainerStopParameters(), CancellationToken.None))
{
    //delete container
    await _client.Containers.RemoveContainerAsync(_containerResponse.ID, new ContainerRemoveParameters(), CancellationToken.None);
}

_client?.Dispose();

True